In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df_order_items = pd.read_csv('/content/DNC_order_items_dataset.csv', index_col= 'Unnamed: 0')
df_order_reviews = pd.read_csv('/content/DNC_order_reviews_dataset.csv', index_col= 'Unnamed: 0')
df_orders = pd.read_csv('/content/DNC_orders_dataset.csv', index_col= 'Unnamed: 0')
df_products = pd.read_csv('/content/DNC_products_dataset.csv', index_col= 'Unnamed: 0')

Definição do Modelo -> Com a base de dados disponibilizada, quais modelos de recomendação podem ser feitos? 
De acordo com a exploratória dos dados, pode ser feito modelos de recomendação de similaridade de produtos, no "cold start" do cliente, irei recomendar os produtos mais pedidos e nota acima de 4, para que de acordo com a navegação do cliente seja recomendado produtos similares aos que ele procura



#EDA

In [3]:
print(df_order_items.shape)
print(df_order_reviews.shape)
print(df_orders.shape)
print(df_products.shape)

(112650, 4)
(99224, 3)
(99441, 3)
(32951, 8)


In [4]:
print((df_order_items.isnull().sum()/df_order_items.shape[0])*100)
print('------------------')
print((df_order_reviews.isnull().sum()/df_order_reviews.shape[0])*100)
print('------------------')
print((df_orders.isnull().sum()/df_orders.shape[0])*100)
print('---------------------------------')
print((df_products.isnull().sum()/df_products.shape[0])*100)

order_id         0.0
order_item_id    0.0
product_id       0.0
price            0.0
dtype: float64
------------------
review_id       0.0
order_id        0.0
review_score    0.0
dtype: float64
------------------
order_id        0.0
customer_id     0.0
order_status    0.0
dtype: float64
---------------------------------
product_id                    0.000000
product_category_name         1.851234
product_name_lenght           1.851234
product_description_lenght    1.851234
product_weight_g              0.006070
product_length_cm             0.006070
product_height_cm             0.006070
product_width_cm              0.006070
dtype: float64


In [5]:
df_order_items.describe().T

,count,mean,std,min,25%,50%,75%,max
order_item_id,112650.0,1.197834,0.705124,1.00,1.0,1.00,1.0,21.0
price,112650.0,120.653739,183.633928,0.85,39.9,74.99,134.9,6735.0


In [6]:
df_order_reviews.describe().T

,count,mean,std,min,25%,50%,75%,max
review_score,99224.0,4.086421,1.347579,1.0,4.0,5.0,5.0,5.0


In [7]:
df_orders.describe().T

,count,unique,top,freq
order_id,99441,99441,52f89521400b32703e434cd646403475,1
customer_id,99441,99441,e7232b4d5318177ea3ab911dfb920cbf,1
order_status,99441,8,delivered,96478


In [8]:
df_products.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
product_id,32951,32951,8e3a9b154a86602302ce789b9eb8b6c7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_category_name,32341,73,cama_mesa_banho,3029,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_name_lenght,32341,NaN,NaN,NaN,48.4769,10.2457,5,42,51,57,76
product_description_lenght,32341,NaN,NaN,NaN,771.495,635.115,4,339,595,972,3992
product_weight_g,32949,NaN,NaN,NaN,2276.47,4282.04,0,300,700,1900,40425
product_length_cm,32949,NaN,NaN,NaN,30.8151,16.9145,7,18,25,38,105
product_height_cm,32949,NaN,NaN,NaN,16.9377,13.6376,2,8,13,21,105
product_width_cm,32949,NaN,NaN,NaN,23.1967,12.079,6,15,20,30,118


In [9]:
df_products.dropna(inplace=True)

In [10]:
df_products.shape

(32340, 8)

#Produtos com maior quantidade de vendas e com notas maior ou igual a 4 para o Cold Start

In [11]:
df1 = df_order_items.merge(df_products)
df = df1.merge(df_order_reviews)

In [12]:
df.shape

(110773, 13)

In [13]:
df.isnull().sum()

order_id                      0
order_item_id                 0
product_id                    0
price                         0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_weight_g              0
product_length_cm             0
product_height_cm             0
product_width_cm              0
review_id                     0
review_score                  0
dtype: int64

In [14]:
df_filter = df[df['review_score'] >= 4]

In [15]:
df_filter.head()

,order_id,order_item_id,product_id,price,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm,review_id,review_score
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,97ca439bc427b48bc1cd7177abe71365,5
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,55.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,b11cba360bbe71410c291b764753d37f,5
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,64.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,af01c4017c5ab46df6cc810e069e654a,4
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,8304ff37d8b16b57086fa283fe0c44f8,5
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0,426f43a82185969503fb3c86241a9535,5


In [16]:
df_filter.columns

Index(['order_id', 'order_item_id', 'product_id', 'price',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'review_id', 'review_score'],
      dtype='object')

In [17]:
columns = ['order_id', 'order_item_id', 'price', 'product_name_lenght',
       'product_description_lenght', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'review_id', 'review_score']
df_filter.drop(columns=columns, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [18]:
df_filter.head()

,product_id,product_category_name
0,4244733e06e7ecb4970a6e2683c13e61,cool_stuff
1,4244733e06e7ecb4970a6e2683c13e61,cool_stuff
2,4244733e06e7ecb4970a6e2683c13e61,cool_stuff
3,4244733e06e7ecb4970a6e2683c13e61,cool_stuff
4,4244733e06e7ecb4970a6e2683c13e61,cool_stuff


In [19]:
def cold_start(dataf,n=5):
  return dataf.groupby(by=['product_category_name','product_id'], as_index='product_category_name') \
    .agg({'product_id': 'count'}) \
    .rename(columns={'product_id': 'product_id_count'}) \
    .sort_values(['product_category_name','product_id_count'], ascending = False) \
    .groupby(by='product_category_name').head(n).reset_index()

In [20]:
df_top_5 = cold_start(df_filter)

In [21]:
df_top_5

,product_category_name,product_id,product_id_count
0,utilidades_domesticas,42a2c92a0979a949ca4ea89ec5c7b934,129
1,utilidades_domesticas,f7a17d2c51d9df89a4f1711c4ac17f33,59
2,utilidades_domesticas,5a6e53c3b4e8684b13388d6aa4afdf12,57
3,utilidades_domesticas,d696750e550fd0f733979dd7e5dff921,51
4,utilidades_domesticas,e9a69340883a438c3f91739d14d3a56d,43
...,...,...,...
348,agro_industria_e_comercio,11250b0d4b709fee92441c5f34122aed,22
349,agro_industria_e_comercio,423a6644f0aa529e8828ff1f91003690,16
350,agro_industria_e_comercio,672e757f331900b9deea127a2a7b79fd,15
351,agro_industria_e_comercio,3bebad3cf2c8d1a8d3ce97174643e054,10


#Recomendação dos produtos

In [22]:
df_recomend = df.drop(columns=['order_id', 'order_item_id','review_id'])

In [23]:
numeric = ['product_name_lenght','product_description_lenght', 'product_weight_g',
           'product_length_cm','product_height_cm', 'product_width_cm', 'price', 'review_score']
categ = ['product_id','product_category_name']

In [24]:
df_recomend = df_recomend.groupby(by=categ, as_index=False)[numeric].median()

In [25]:
df_sample = df_recomend.sample(2000)
df_sample = df_sample.set_index('product_id')
df_sample = pd.get_dummies(df_sample)

In [26]:
prod_recom =  cosine_similarity(df_sample)

In [27]:
prod_recom

array([[1.        , 0.57327115, 0.96137501, ..., 0.96616909, 0.94102737,
        0.95718884],
       [0.57327115, 1.        , 0.77261887, ..., 0.34465814, 0.28276606,
        0.7562827 ],
       [0.96137501, 0.77261887, 1.        , ..., 0.8593864 , 0.82413581,
        0.99316554],
       ...,
       [0.96616909, 0.34465814, 0.8593864 , ..., 1.        , 0.98737488,
        0.85772689],
       [0.94102737, 0.28276606, 0.82413581, ..., 0.98737488, 1.        ,
        0.83918423],
       [0.95718884, 0.7562827 , 0.99316554, ..., 0.85772689, 0.83918423,
        1.        ]])

In [28]:
prod_semelhantes = df_sample.index[20]

print(f'O produto escolhido é: {prod_semelhantes}')

prod_id = df_sample.index.tolist().index(prod_semelhantes)
print(f'Produto: {prod_semelhantes}, tem índice: {prod_id}')

similares_10 = np.argsort(-prod_recom[prod_id])[1:10]

for i in zip(df_sample.index[similares_10],prod_recom[prod_id][similares_10]):
  print(f'Produto {i[0]} tem similaridade {i[1]:2f} com produto {prod_semelhantes}')

O produto escolhido é: 740867136b4bf7605e36c5913b91eb99
Produto: 740867136b4bf7605e36c5913b91eb99, tem índice: 20


NameError: ignored